Implementation of <i>Learning a similarity metric discriminatively, with application to face verification</i>, by Sumit Chopra, Raia Hadsell and Yann LeCun (2005).  Cited by 405 at the time of this writing (21 July 2016). <br><br>

Code structure borrowed from the Keras examples: https://github.com/fchollet/keras/tree/master/examples
 
The best performing Siamese ConvNet architecture is: C1, S2, C3, S4, C5, F6. C is a convolution layer, S is a "sub-sampling" (probably a.k.a. pooling?) layer, F is a fully connected layer.

In [13]:
%matplotlib inline
import numpy as np
import random
import time

from keras.datasets import cifar10
from keras.models import Sequential, Model
#from keras.models import load_model. Why can't this import?
from keras.layers import Dense, Dropout, Activation, Merge, Embedding, Lambda
from keras.layers import Convolution2D, MaxPooling2D, Layer, Input
from keras.layers import Flatten
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras import backend as K

import theano
import theano.tensor as T

import skynet.utils as utils

ImportError: cannot import name 'load_model'

In [2]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i+1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq


def create_base_convnet(input_img_shape, verbose=False):
    cnn = Sequential()
    #C1
    cnn.add(Convolution2D(15, 7, 7, 
                          activation='relu', 
                          border_mode='same',
                          input_shape=input_img_shape))
    if verbose:
        print("C1 shape: ", cnn.output_shape)

    #S2
    cnn.add(MaxPooling2D((2, 2), border_mode='same'))
    if verbose:
        print("S2 shape: ", cnn.output_shape)

    #C3
    cnn.add(Convolution2D(45, 6, 6, 
                          activation='relu', 
                          border_mode='same'))
    if verbose:
        print("C3 shape: ", cnn.output_shape)

    #S4
    cnn.add(MaxPooling2D((4,3), border_mode='same'))
    if verbose:
        print("S4 shape: ", cnn.output_shape)
    #C5
    cnn.add(Convolution2D(250, 5, 5,
                          activation='relu',
                          border_mode='same'))
    if verbose:
        print("C5 shape: ", cnn.output_shape)
    #F6
    cnn.add(Flatten())
    cnn.add(Dense(50))
    if verbose:
        print("F6 shape: ", cnn.output_shape)

    return cnn


def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()


In [3]:
# Load bbbc021 patch data
# Warning: these are MOA labels!
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('label_train.npy')
y_test = np.load('label_test.npy')

n_trg, patchlen, patchlen, ch = x_train.shape
n_test, patchlen, patchlen, ch = x_test.shape

x_train = utils.swap_image_format_list(x_train)
x_test = utils.swap_image_format_list(x_test)

print("Data Stats")
print("-"*10)
print("x_train.shape = %s" % (x_train.shape,))
print("x_test.shape = %s" % (x_test.shape,))

labels = list(set(y_train))

#Convert y_labels from string to integer categories
y_train2 = utils.my_one_hot_index(y_train, labels)
y_test2 = utils.my_one_hot_index(y_test, labels)

# Set params
nb_classes = 13
input_img_shape = (3, 30, 30)

Data Stats
----------
x_train.shape = (76960, 3, 30, 30)
x_test.shape = (19240, 3, 30, 30)


In [4]:
# Generate genuine and imposter input pairs
digit_indices = [np.where(y_train2 == i)[0] for i in range(10)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test2 == i)[0] for i in range(10)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

print("No. of training pairs: ", len(tr_pairs))
print("No. of testing pairs: ", len(te_pairs))

No. of training pairs:  23980
No. of testing pairs:  5980


In [5]:
# network definition
input_a = Input(shape=input_img_shape)
input_b = Input(shape=input_img_shape)

base_network = create_base_convnet(input_img_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(input=[input_a, input_b], output=distance)

In [9]:
# train
nb_epoch=50
batch_size = 32
t0 = time.time()

rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], 
          tr_y,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

dt = time.time() - t0
m, s = divmod(dt, 60)
h, m = divmod(m, 60)
print("Done in %d:%02d:%02d" % (h, m, s))
# batch_size = 128: loss ~= 8.4336, dt ~= 800s
# batch_size = 32: val_loss ~=0.2586, dt = 944s

Train on 23980 samples, validate on 5980 samples
Epoch 1/50
23980/23980 [==============================] - 177s - loss: 0.1394 - val_loss: 0.1585
Epoch 2/50
23980/23980 [==============================] - 161s - loss: 0.1364 - val_loss: 0.1564
Epoch 3/50
23980/23980 [==============================] - 163s - loss: 0.1360 - val_loss: 0.1732
Epoch 4/50
23980/23980 [==============================] - 164s - loss: 0.1339 - val_loss: 0.1728
Epoch 5/50
23980/23980 [==============================] - 164s - loss: 0.1315 - val_loss: 0.1468
Epoch 6/50
23980/23980 [==============================] - 165s - loss: 0.1313 - val_loss: 0.1477
Epoch 7/50
23980/23980 [==============================] - 167s - loss: 0.1296 - val_loss: 0.1568
Epoch 8/50
23980/23980 [==============================] - 178s - loss: 0.1273 - val_loss: 0.1616
Epoch 9/50
23980/23980 [==============================] - 192s - loss: 0.1269 - val_loss: 0.1623
Epoch 10/50
23980/23980 [==============================] - 186s - loss: 0.1257

In [10]:
# compute final accuracy on training and test sets
pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(pred, tr_y)
pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(pred, te_y)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 86.60%
* Accuracy on test set: 80.58%
